In [2]:
!pip install git+https://github.com/huggingface/parler-tts

  Cloning https://github.com/huggingface/parler-tts to /tmp/pip-req-build-a32rrbi2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts /tmp/pip-req-build-a32rrbi2
  Resolved https://github.com/huggingface/parler-tts to commit d108732cd57788ec86bc857d99a6cabd66663d68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-install-6qx1j5fk/descript-audiotools_5e369ddf53df4f15b7fefbd2dc467b14
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-install-6qx1j5fk/descript-audiotools_5e369ddf53df4f15b7fefbd2dc467b14
  Resolved https://github.com/descriptinc/audiotools to commit 348ebf2034ce24e2a91a553e3171cb00c0c71678
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.3 MB/s eta 0:00:00
  Preparing metada

In [3]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.75G [00:00<?, ?B/s]

  "_name_or_path": "google/flan-t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.46.1",
  "use_cache": true,
  "vocab_size": 32128
}

  "_name_or_path": "ylacombe/dac_44khz",
  "architectures": [
    "DacModel"
  ],
  "codebook_dim": 8,
  "codebook_loss_weight": 1.0,
  "codebook_size": 1024,
  "commitment_loss_weight": 0.25,
  "decoder_hidden_si

generation_config.json:   0%|          | 0.00/223 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [4]:
def generate_audio(prompt, description,speaker,language):
    # prompt = "Hey, how are you doing today?"
    # description = "A female speaker with a British accent delivers a slightly expressive and animated speech with a moderate speed and pitch. The recording is of very high quality, with the speaker's voice sounding clear and very close up."
    print(language + "\t" + speaker + "\n" + prompt + "\n" + description)
    description_input_ids = description_tokenizer(description, return_tensors="pt").to(device)
    prompt_input_ids = tokenizer(prompt, return_tensors="pt").to(device)
    print("Debug: Generting...")
    generation = model.generate(input_ids=description_input_ids.input_ids, attention_mask=description_input_ids.attention_mask, prompt_input_ids=prompt_input_ids.input_ids, prompt_attention_mask=prompt_input_ids.attention_mask)
    audio_arr = generation.cpu().numpy().squeeze()
    print("Debug: Saving...")
    sf.write(f"output_{language}_{speaker}.wav", audio_arr, model.config.sampling_rate)
    print(f"Debug: ✅ Audio saved: output_{language}_{speaker}.wav")

In [5]:
languages_texts = {


    "mar": [
        "ग्राहक सेवा केंद्रात कॉल केल्याबद्दल धन्यवाद. [screaming] मी आज तुम्हाला कशी मदत करू शकतो? मला-- तुमची अडचण समजली आहे, [crying] आणि मी तुम्हाला मदत करण्यासाठी येथे आहे.",
        "Sanjay's voice is clear sound, close sound, expressive, with rise and fall intonation, variable speed"]
}

    # "knd": [
    #     "ಗ್ರಾಹಕ ಸೇವೆಗೆ ಕರೆ ಮಾಡಿದ ಧನ್ಯವಾದಗಳು. [laughs] ನಾನು ನಿಮಗೆ ಹೇಗೆ ಸಹಾಯ ಮಾಡಬಹುದು? [angry] ನಿಮ್ಮ ಚಿಂತೆಯನ್ನು ನಾನು ಅರ್ಥಮಾಡಿಕೊಂಡಿದ್ದೇನೆ, ನಾನು ಸಹಾಯ ಮಾಡಲು ಇಲ್ಲಿದ್ದೇನೆ.",
    #     "Suresh has a clear sound, close sound, expressive, with rise and fall intonation."
    # ],
    # "knd": [
    #     "ಗ್ರಾಹಕ ಸೇವೆಗೆ ಕರೆ ಮಾಡಿದ ಧನ್ಯವಾದಗಳು. ನಾನು ನಿಮಗೆ ಹೇಗೆ ಸಹಾಯ ಮಾಡಬಹುದು? ನಿಮ್ಮ ಚಿಂತೆಯನ್ನು ನಾನು ಅರ್ಥಮಾಡಿಕೊಂಡಿದ್ದೇನೆ, ನಾನು ಸಹಾಯ ಮಾಡಲು ಇಲ್ಲಿದ್ದೇನೆ. ದಯವಿಟ್ಟು ಸ್ವಲ್ಪ ನಿರೀಕ್ಷಿಸಿ, ನಾನು ಆ ಮಾಹಿತಿಯನ್ನು ಪರಿಶೀಲಿಸುತ್ತಿದ್ದೇನೆ. ನಿಮ್ಮ ಸಮಸ್ಯೆಯನ್ನು ಸಂಬಂಧಿತ ವಿಭಾಗಕ್ಕೆ ಕಳುಹಿಸಲಾಗಿದೆ. ಉಂಟಾದ ತೊಂದರೆಗಾಗಿ ವಿಷಾದಿಸುತ್ತೇವೆ.",
    #     "Suresh's voice is steady and confident, spoken at a moderate speed with a clear and professional delivery."
    # ],
    # "mar": [
    #     "    ग्राहक सेवा केंद्रात कॉल केल्याबद्दल धन्यवाद. मी आज तुम्हाला कशी मदत करू शकतो? मला तुमची अडचण समजली आहे, आणि मी तुम्हाला मदत करण्यासाठी येथे आहे.कृपया थांबा, मी ही माहिती तपासत आहे. तुमची समस्या संबंधित विभागाकडे पाठवली आहे. झालेल्या गैरसोयीबद्दल आम्ही दिलगीर आहोत.",
    #     "Sanjay's voice is warm and courteous, featuring slow, deliberate enunciation and a noise-free background ideal for support calls."
    # ]
    # "eng": [
    #     "Thank you for calling customer support. How can I help you today? I understand your concern, and I’m here to assist you. Please hold the line while I check that information for you. Your issue has been escalated to the concerned department. We apologize for the inconvenience caused.",
    #     "Thoma's voice is calm, well-articulated, and maintains a consistent medium pace with minimal background noise, ideal for customer support."
    # ],
    # "hin": [
    #     "ग्राहक सेवा में कॉल करने के लिए धन्यवाद। मैं आज आपकी कैसे सहायता कर सकता हूँ? मैं आपकी समस्या को समझता हूँ, और आपकी मदद के लिए यहाँ हूँ। कृपया लाइन पर बने रहें, मैं आपके लिए जानकारी देख रहा हूँ। आपकी समस्या को संबंधित विभाग को भेज दिया गया है। हुई असुविधा के लिए हम क्षमा चाहते हैं।",
    #     "Rohit's voice is polite and reassuring, with clear pronunciation and a gentle tone suitable for handling customer concerns."
    # ],

In [ ]:
# prompt: use generate_tts_mp3 to generate auido for language_texts one by one

import time
start_time = time.time()

for lang, texts in languages_texts.items():
    text = texts[0]
    description = texts[1]
    speaker_name = description.split()[0]
    generate_audio(text, description, lang, speaker_name)
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total time taken: {total_time} seconds")

Sanjay's	mar
ग्राहक सेवा केंद्रात कॉल केल्याबद्दल धन्यवाद. [screaming] मी आज तुम्हाला कशी मदत करू शकतो? मला-- तुमची अडचण समजली आहे, [crying] आणि मी तुम्हाला मदत करण्यासाठी येथे आहे.
Sanjay's voice is clear sound, close sound, expressive, with rise and fall intonation, variable speed
Debug: Generting...


#### Bullsh*t code

In [ ]:
!pip install git+https://github.com/huggingface/parler-tts.git
!pip install pydub mutagen
!pip install git+https://github.com/huggingface/transformers.git

In [ ]:
!pip install transformers==4.46.1

In [ ]:
import os
os.kill(os.getpid(), 9)  # force-restarts the runtime

Model page: https://huggingface.co/ai4bharat/indic-parler-tts

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/ai4bharat/indic-parler-tts)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration

from transformers import pipeline
# Now use the pipeline
pipe = pipeline("text-to-speech", model="ai4bharat/indic-parler-tts")


In [ ]:
# Load model directly
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indic-parler-tts")

In [ ]:
from transformers import AutoTokenizer
import soundfile as sf
from pydub import AudioSegment
from mutagen.easyid3 import EasyID3
import os

# Load models once globally
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

def generate_tts_mp3(text, description, language_code, speaker_name):
    # Tokenize inputs
    description_ids = description_tokenizer(description, return_tensors="pt").to(device)
    text_ids = tokenizer(text, return_tensors="pt").to(device)

    # Generate speech
    generation = model.generate(
        input_ids=description_ids.input_ids,
        attention_mask=description_ids.attention_mask,
        prompt_input_ids=text_ids.input_ids,
        prompt_attention_mask=text_ids.attention_mask
    )

    # Convert to numpy and save as temporary WAV
    audio_arr = generation.cpu().numpy().squeeze()
    wav_path = f"{language_code}_{speaker_name}.wav"
    sf.write(wav_path, audio_arr, model.config.sampling_rate)

    # Convert to MP3
    mp3_path = f"{language_code}_{speaker_name}.mp3"
    audio = AudioSegment.from_wav(wav_path)
    audio.export(mp3_path, format="mp3")

    # Add metadata
    audio_file = EasyID3(mp3_path)
    audio_file["title"] = f"TTS by {speaker_name}"
    audio_file["artist"] = speaker_name
    audio_file["language"] = language_code
    audio_file["comment"] = description
    audio_file.save()

    # Remove temporary WAV file
    os.remove(wav_path)

    print(f"✅ MP3 saved: {mp3_path} with metadata.")

# Example usage:
# generate_tts_mp3("अरे, तुम आज कैसे हो?", "Divya's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise.", "hin", "Divya")